This notebook shows how to use optimum-benchmark to benchmark LLMs. It  focuses on benchmarking for Phi-3.

We need to install the following packages. 

In [1]:
!python -m pip install git+https://github.com/huggingface/optimum-benchmark.git
!pip install vllm
!pip install optimum-benchmark[vllm]

  Cloning https://github.com/huggingface/optimum-benchmark.git to /tmp/pip-req-build-8wpm6fcw
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/optimum-benchmark.git /tmp/pip-req-build-8wpm6fcw
  Resolved https://github.com/huggingface/optimum-benchmark.git to commit 347e13ca9f7f904f55669603cfb9f0b6c7e8672c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done


Define the configuration for optimum-benchmark.

Here we benchmark for inference, using different batch sizes, Mistral 7B loaded as fp16.
If you run this notebook on Google Colab, you will need the A100 only for this part. The following benchmarks would run on the T4.

In [1]:
!nvidia-smi

Tue Jun 11 08:38:13 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA H100 NVL                Off | 00000001:00:00.0 Off |                    0 |
| N/A   36C    P0              60W / 400W |      7MiB / 95830MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [11]:
from optimum_benchmark import Benchmark, BenchmarkConfig, TorchrunConfig, InferenceConfig, PyTorchConfig  
from optimum_benchmark.logging_utils import setup_logging  
  
# setup_logging(level="INFO", handlers=["console"])  
if __name__ == "__main__":  
    launcher_config = TorchrunConfig(nproc_per_node=1)  
      
    # 设置推理配置，包括批处理大小和序列长度等参数  
    scenario_config = InferenceConfig(  
        latency=True,  
        memory=True,  
        warmup_runs=5,  
        input_shapes={  
            "batch_size": 1,  # 设置批处理大小  
            "sequence_length": 512  # 设置序列长度  
        },  
        generate_kwargs={  
            "max_new_tokens": 100,  # 默认生成的新标记的最大数量  
            "min_new_tokens": 100,  # 默认生成的新标记的最小数量  
            "num_beams": 1,  # 默认束搜索数量  
            "temperature": 1.0,  # 默认温度  
            "top_k": 50,  # 默认Top-k采样  
            "top_p": 0.9,  # 默认Top-p采样  
            "do_sample": True  # 启用采样模式 
        }  
    ) 
      
    # 定义 PyTorch 后端配置  
    backend_config = PyTorchConfig(  
        model="microsoft/Phi-3-mini-4k-instruct",  
        device="cuda",  
        device_ids="0",  # 确保设备ID为字符串  
        no_weights=True  
    )  
      
    # 定义基准测试配置  
    benchmark_config = BenchmarkConfig(  
        name="pytorch_Phi-3-mini-4k-instruct",  
        scenario=scenario_config,  
        launcher=launcher_config,  
        backend=backend_config,  
    )  
      
    # 运行基准测试  
    benchmark_report = Benchmark.launch(benchmark_config)  
      
    # 在终端中记录基准测试结果  
    benchmark_report.log()  # or print(benchmark_report)  
      
    # 将工件转换为字典或数据帧  
    benchmark_config.to_dict()  # or benchmark_config.to_dataframe()  
      
    # 将工件保存到磁盘为 JSON 或 CSV 文件  
    benchmark_report.save_csv("benchmark_report_pytorch_Phi-3-mini-4k-instruct.csv")  # 保存为 CSV 文件  
    benchmark_report.save_json("benchmark_report_pytorch_Phi-3-mini-4k-instruct.json")  # 保存为 JSON 文件  
      
    # 或者将它们合并到一个单一的工件中  
    benchmark = Benchmark(config=benchmark_config, report=benchmark_report)  
    benchmark.save_json("benchmark_pytorch_Phi-3-mini-4k-instruct.json")  # 保存为 JSON 文件  
    benchmark.save_csv("benchmark_pytorch_Phi-3-mini-4k-instruct.csv")  # 保存为 CSV 文件  


[ISOLATED-PROCESS][2024-06-11 10:59:23,322][torchrun][INFO] - 	+ Starting benchmark in isolated process
[RANK-PROCESS-0][2024-06-11 10:59:25,391][torchrun][INFO] - 	+ Setting torch.distributed cuda device to 0
[RANK-PROCESS-0][2024-06-11 10:59:25,394][torchrun][INFO] - 	+ Initializing torch.distributed process group
[RANK-PROCESS-0][2024-06-11 10:59:25,415][datasets][INFO] - PyTorch version 2.3.0 available.
[RANK-PROCESS-0][2024-06-11 10:59:25,591][pytorch][INFO] - Allocating pytorch backend
[RANK-PROCESS-0][2024-06-11 10:59:25,591][pytorch][INFO] - 	+ Benchmarking a Transformers model


[W ProcessGroupGloo.cpp:721] Warning: Unable to resolve hostname to a (local) address. Using the loopback address as fallback. Manually set the network interface to bind to with GLOO_SOCKET_IFNAME. (function operator())
/opt/miniconda/envs/benchmark/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


[RANK-PROCESS-0][2024-06-11 10:59:26,330][pytorch][INFO] - 	+ Using automodel class AutoModelForCausalLM
[RANK-PROCESS-0][2024-06-11 10:59:26,330][pytorch][INFO] - 	+ Seeding pytorch backend with seed 42
[RANK-PROCESS-0][2024-06-11 10:59:26,335][pytorch][INFO] - 	+ Using AutoModel AutoModelForCausalLM
[RANK-PROCESS-0][2024-06-11 10:59:26,335][pytorch][INFO] - 	+ Creating backend temporary directory
[RANK-PROCESS-0][2024-06-11 10:59:26,335][pytorch][INFO] - 	+ Loading model with random weights
[RANK-PROCESS-0][2024-06-11 10:59:26,336][pytorch][INFO] - 	+ Creating no weights model directory
[RANK-PROCESS-0][2024-06-11 10:59:26,336][pytorch][INFO] - 	+ Creating no weights model state dict
[RANK-PROCESS-0][2024-06-11 10:59:26,353][pytorch][INFO] - 	+ Saving no weights model safetensors
[RANK-PROCESS-0][2024-06-11 10:59:26,354][pytorch][INFO] - 	+ Saving no weights model pretrained config
[RANK-PROCESS-0][2024-06-11 10:59:26,354][pytorch][INFO] - 	+ Loading Transformers model using device c

In [13]:
#Limit gpu_memory_utilization in vLLM
import os  
import torch  
from transformers import AutoModelForCausalLM, AutoTokenizer  
from optimum_benchmark import Benchmark, BenchmarkConfig, TorchrunConfig, InferenceConfig, VLLMConfig  
from optimum_benchmark.logging_utils import setup_logging  

# setup_logging(level="INFO", handlers=["console"])  
if __name__ == "__main__":  
    launcher_config = TorchrunConfig(nproc_per_node=1)  
      
    # 设置推理配置，包括batch size、sequence length等参数  
    scenario_config = InferenceConfig(  
        latency=True,  
        memory=True,  
        warmup_runs=5,  
        input_shapes={  
            "batch_size": 1,  # 默认批处理大小  
            "sequence_length": 512,  # 默认序列长度 
            "max_sequence_length": 4096
        },  
        generate_kwargs={  
            "max_new_tokens": 50,  # 默认生成的新标记的最大数量  
            "min_new_tokens": 50,  # 默认生成的新标记的最小数量  
            "num_beams": 1,  # 默认束搜索数量  
            "temperature": 1.0,  # 默认温度  
            "top_k": 50,  # 默认Top-k采样  
            "top_p": 0.9,  # 默认Top-p采样  
            "do_sample": True  # 启用采样模式 
        }  
    )  
      
    # 定义 vllm_config  
    vllm_config = VLLMConfig(  
        model="microsoft/Phi-3-mini-4k-instruct",  
        device="cuda",  
        device_ids="0",  # 确保设备ID为字符串  
        no_weights=True,  
        gpu_memory_utilization=0.20
    )  
      
    benchmark_config = BenchmarkConfig(  
        name="vllm_phi3_mini_4k_instruct",  
        scenario=scenario_config,  
        launcher=launcher_config,  
        backend=vllm_config,  
    )  
      
    benchmark_report = Benchmark.launch(benchmark_config)  
      
    # log the benchmark in terminal  
    benchmark_report.log()  # or print(benchmark_report)  
      
    # convert artifacts to a dictionary or dataframe  
    benchmark_config.to_dict()  # or benchmark_config.to_dataframe()  
      
    # save artifacts to disk as json or csv files  
    benchmark_report.save_csv("benchmark_reportvllm.Phi-3-mini-4k-instruct.csv")  # 保存为 CSV 文件  
    benchmark_report.save_json("benchmark_reportvllm.Phi-3-mini-4k-instruct.json")  # 保存为 JSON 文件  
      
    # or merge them into a single artifact  
    benchmark = Benchmark(config=benchmark_config, report=benchmark_report)  
    benchmark.save_json("benchmarkvllm.Phi-3-mini-4k-instruct.json")  # 保存为 JSON 文件  
    benchmark.save_csv("benchmarkvllm.Phi-3-mini-4k-instruct.csv")  # 保存为 CSV 文件  


[ISOLATED-PROCESS][2024-06-11 11:04:28,373][torchrun][INFO] - 	+ Starting benchmark in isolated process
[RANK-PROCESS-0][2024-06-11 11:04:30,484][torchrun][INFO] - 	+ Setting torch.distributed cuda device to 0
[RANK-PROCESS-0][2024-06-11 11:04:30,488][torchrun][INFO] - 	+ Initializing torch.distributed process group


[W ProcessGroupGloo.cpp:721] Warning: Unable to resolve hostname to a (local) address. Using the loopback address as fallback. Manually set the network interface to bind to with GLOO_SOCKET_IFNAME. (function operator())


[RANK-PROCESS-0][2024-06-11 11:04:30,723][datasets][INFO] - PyTorch version 2.3.0 available.
INFO 06-11 11:04:30 base.py:41] Allocating vllm backend
INFO 06-11 11:04:30 base.py:60] 	+ Benchmarking a Transformers model


/opt/miniconda/envs/benchmark/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


INFO 06-11 11:04:31 base.py:74] 	+ Using automodel class AutoModelForCausalLM
INFO 06-11 11:04:31 base.py:78] 	+ Seeding vllm backend with seed 42
INFO 06-11 11:04:31 backend.py:23] 	+ Creating backend temporary directory
INFO 06-11 11:04:31 backend.py:27] 	+ Loading no weights model
INFO 06-11 11:04:31 backend.py:90] 	+ Creating no weights model
INFO 06-11 11:04:31 backend.py:59] 	+ Creating no weights model directory
INFO 06-11 11:04:31 backend.py:61] 	+ Creating no weights model state dict
INFO 06-11 11:04:31 backend.py:63] 	+ Saving no weights model safetensors
INFO 06-11 11:04:31 backend.py:66] 	+ Saving no weights model pretrained config
INFO 06-11 11:04:31 backend.py:68] 	+ Saving no weights model pretrained processor
INFO 06-11 11:04:31 backend.py:71] 	+ Loading no weights model from /tmp/tmpimi46wzo/no_weights_model
[RANK-PROCESS-0][2024-06-11 11:04:31,343][accelerate.utils.modeling][INFO] - We will use 90% of the memory on device 0 for storing the model, and 10% for the buffe

In [14]:
#Do not limit gpu_memory_utilization in vLLM
import os  
import torch  
from transformers import AutoModelForCausalLM, AutoTokenizer  
from optimum_benchmark import Benchmark, BenchmarkConfig, TorchrunConfig, InferenceConfig, VLLMConfig  
from optimum_benchmark.logging_utils import setup_logging  

# setup_logging(level="INFO", handlers=["console"])  
if __name__ == "__main__":  
    launcher_config = TorchrunConfig(nproc_per_node=1)  
      
    # 设置推理配置，包括batch size、sequence length等参数  
    scenario_config = InferenceConfig(  
        latency=True,  
        memory=True,  
        warmup_runs=5,  
        input_shapes={  
            "batch_size": 1,  # 默认批处理大小  
            "sequence_length": 512,  # 默认序列长度 
            "max_sequence_length": 4096
        },  
        generate_kwargs={  
            "max_new_tokens": 50,  # 默认生成的新标记的最大数量  
            "min_new_tokens": 50,  # 默认生成的新标记的最小数量  
            "num_beams": 1,  # 默认束搜索数量  
            "temperature": 1.0,  # 默认温度  
            "top_k": 50,  # 默认Top-k采样  
            "top_p": 0.9,  # 默认Top-p采样  
            "do_sample": True  # 启用采样模式 
        }  
    )  
      
    # 定义 vllm_config  
    vllm_config = VLLMConfig(  
        model="microsoft/Phi-3-mini-4k-instruct",  
        device="cuda",  
        device_ids="0",  # 确保设备ID为字符串  
        no_weights=True,  
    #    gpu_memory_utilization=0.20
    )  
      
    benchmark_config = BenchmarkConfig(  
        name="vllm_phi3_mini_4k_instruct",  
        scenario=scenario_config,  
        launcher=launcher_config,  
        backend=vllm_config,  
    )  
      
    benchmark_report = Benchmark.launch(benchmark_config)  
      
    # log the benchmark in terminal  
    benchmark_report.log()  # or print(benchmark_report)  
      
    # convert artifacts to a dictionary or dataframe  
    benchmark_config.to_dict()  # or benchmark_config.to_dataframe()  
      
    # save artifacts to disk as json or csv files  
    benchmark_report.save_csv("benchmark_reportvllm.Phi-3-mini-4k-instruct.csv")  # 保存为 CSV 文件  
    benchmark_report.save_json("benchmark_reportvllm.Phi-3-mini-4k-instruct.json")  # 保存为 JSON 文件  
      
    # or merge them into a single artifact  
    benchmark = Benchmark(config=benchmark_config, report=benchmark_report)  
    benchmark.save_json("benchmarkvllm.Phi-3-mini-4k-instruct.json")  # 保存为 JSON 文件  
    benchmark.save_csv("benchmarkvllm.Phi-3-mini-4k-instruct.csv")  # 保存为 CSV 文件  


[ISOLATED-PROCESS][2024-06-11 11:09:18,896][torchrun][INFO] - 	+ Starting benchmark in isolated process
[RANK-PROCESS-0][2024-06-11 11:09:20,987][torchrun][INFO] - 	+ Setting torch.distributed cuda device to 0
[RANK-PROCESS-0][2024-06-11 11:09:20,990][torchrun][INFO] - 	+ Initializing torch.distributed process group


[W ProcessGroupGloo.cpp:721] Warning: Unable to resolve hostname to a (local) address. Using the loopback address as fallback. Manually set the network interface to bind to with GLOO_SOCKET_IFNAME. (function operator())


[RANK-PROCESS-0][2024-06-11 11:09:21,219][datasets][INFO] - PyTorch version 2.3.0 available.
INFO 06-11 11:09:21 base.py:41] Allocating vllm backend
INFO 06-11 11:09:21 base.py:60] 	+ Benchmarking a Transformers model


/opt/miniconda/envs/benchmark/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


INFO 06-11 11:09:21 base.py:74] 	+ Using automodel class AutoModelForCausalLM
INFO 06-11 11:09:21 base.py:78] 	+ Seeding vllm backend with seed 42
INFO 06-11 11:09:21 backend.py:23] 	+ Creating backend temporary directory
INFO 06-11 11:09:21 backend.py:27] 	+ Loading no weights model
INFO 06-11 11:09:21 backend.py:90] 	+ Creating no weights model
INFO 06-11 11:09:21 backend.py:59] 	+ Creating no weights model directory
INFO 06-11 11:09:21 backend.py:61] 	+ Creating no weights model state dict
INFO 06-11 11:09:21 backend.py:63] 	+ Saving no weights model safetensors
INFO 06-11 11:09:21 backend.py:66] 	+ Saving no weights model pretrained config
INFO 06-11 11:09:21 backend.py:68] 	+ Saving no weights model pretrained processor
INFO 06-11 11:09:21 backend.py:71] 	+ Loading no weights model from /tmp/tmplvhtfiqb/no_weights_model
[RANK-PROCESS-0][2024-06-11 11:09:21,814][accelerate.utils.modeling][INFO] - We will use 90% of the memory on device 0 for storing the model, and 10% for the buffe